In [10]:
import os
import uuid
import pickle

import pandas as pd

import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [36]:
year = 2021
month = 2
taxi_type = 'green'
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

In [37]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [38]:
RUN_ID = os.getenv('RUN_ID', 'aa0767b5c49145e1b52d5cccde7378c3')

def load_model(run_id):
    mlflow.set_tracking_uri('http://localhost:8080')
    logged_model = f'runs:/{RUN_ID}/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)

In [39]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

In [40]:
!ls output/green/

2021-02.parquet


In [42]:
df = pd.read_parquet(output_file)
df

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,d66cb97f-37fb-4588-8217-81fefbde9f2f,2021-02-01 00:34:03,130,205,17.916667,18.168946,-0.252279,aa0767b5c49145e1b52d5cccde7378c3
1,627436e0-4b15-4f49-8d70-cb79f892fc25,2021-02-01 00:04:00,152,244,6.500000,7.290331,-0.790331,aa0767b5c49145e1b52d5cccde7378c3
2,a12554d1-83c5-4bcc-8c9f-774b4ea896f3,2021-02-01 00:18:51,152,48,15.250000,21.278875,-6.028875,aa0767b5c49145e1b52d5cccde7378c3
3,0209fe7e-3c21-4c87-934b-ee3e8c418d1e,2021-02-01 00:53:27,152,241,18.233333,25.008761,-6.775428,aa0767b5c49145e1b52d5cccde7378c3
4,d678bd3e-d01c-46a0-91cf-34722c2f73b1,2021-02-01 00:57:46,75,42,8.966667,9.380744,-0.414077,aa0767b5c49145e1b52d5cccde7378c3
...,...,...,...,...,...,...,...,...
61916,23aa26de-5942-4ead-b3a6-715cd957bf19,2021-02-28 22:19:00,129,7,10.000000,14.549670,-4.549670,aa0767b5c49145e1b52d5cccde7378c3
61917,5ada7498-8404-4b5a-b8dc-17cbb7610a50,2021-02-28 23:18:00,116,166,9.000000,9.886653,-0.886653,aa0767b5c49145e1b52d5cccde7378c3
61918,a9e5a996-649b-4bbf-9344-917513888c35,2021-02-28 23:44:00,74,151,14.000000,13.597153,0.402847,aa0767b5c49145e1b52d5cccde7378c3
61919,49f1db9c-af62-4fc6-b519-a4dd1a39b7ff,2021-02-28 23:07:00,42,42,7.000000,6.705995,0.294005,aa0767b5c49145e1b52d5cccde7378c3
